**Modulo 5 : Regularización y Dropout**
* Instructor: [Juan Maniglia](https://juanmaniglia.github.io)

# Part 5.4: Drop Out para que Keras disminuya _Overfitting_

Hinton, Srivastava, Krizhevsky, Sutskever y Salakhutdinov (2012) introdujeron el algoritmo de regularización de la deserción. Aunque el abandono funciona de manera diferente que L1 y L2, logra el mismo objetivo: la prevención del sobreajuste. Sin embargo, el algoritmo realiza la tarea eliminando neuronas y conexiones, al menos temporalmente. A diferencia de L1 y L2, no se agrega penalización de peso. Dropout no busca directamente entrenar pesos pequeños.
El abandono funciona haciendo que las neuronas ocultas de la red neuronal no estén disponibles durante parte del entrenamiento. La eliminación de parte de la red neuronal hace que la parte restante se entrene para lograr una buena puntuación incluso sin las neuronas eliminadas. Esto disminuye la coadaptación entre neuronas, lo que resulta en menos sobreajuste.

La mayoría de los marcos de redes neuronales implementan el abandono como una capa separada. Las capas de abandono funcionan como una capa de red neuronal regular densamente conectada. La única diferencia es que las capas de abandono perderán periódicamente algunas de sus neuronas durante el entrenamiento. Puede usar capas de abandono en redes neuronales de alimentación directa regulares.

El programa implementa una capa de abandono como una capa densa que puede eliminar algunas de sus neuronas. Contrariamente a la creencia popular sobre la capa de abandono, el programa no elimina de forma permanente estas neuronas desechadas. Una capa de abandono no pierde ninguna de sus neuronas durante el proceso de entrenamiento y seguirá teniendo exactamente la misma cantidad de neuronas después del entrenamiento. De esta manera, el programa solo enmascara temporalmente las neuronas en lugar de descartarlas.
La Figura 5. DROPOUT muestra cómo una capa de abandono podría ubicarse con otras capas.

**Figure 5.DROPOUT: Dropout Regularization**
![Dropout Regularization](https://raw.githubusercontent.com/jeffheaton/t81_558_deep_learning/master/images/class_9_dropout.png "Dropout Regularization")

Las neuronas descartadas y sus conexiones se muestran como líneas discontinuas. La capa de entrada tiene dos neuronas de entrada, así como una neurona de polarización. La segunda capa es una capa densa con tres neuronas y una neurona de polarización. La tercera capa es una capa de abandono con seis neuronas regulares a pesar de que el programa ha eliminado el 50% de ellas. Si bien el programa elimina estas neuronas, no las calcula ni las entrena. Sin embargo, la red neuronal final utilizará todas estas neuronas para la salida. Como se mencionó anteriormente, el programa solo descarta temporalmente las neuronas.

Durante las iteraciones de entrenamiento posteriores, el programa elige diferentes conjuntos de neuronas de la capa de abandono. Aunque elegimos una probabilidad de abandono del 50%, la computadora no necesariamente dejará caer tres neuronas. Es como si lanzáramos una moneda al aire para cada una de las neuronas candidatas a abandonar para elegir si esa neurona se abandonó. Debe saber que el programa nunca debe dejar caer la neurona de polarización. Solo las neuronas regulares en una capa de abandono son candidatas.
La implementación del algoritmo de entrenamiento influye en el proceso de descarte de neuronas. El conjunto de abandonos cambia con frecuencia una vez por iteración o lote de entrenamiento. El programa también puede proporcionar intervalos donde todas las neuronas están presentes. Algunos marcos de redes neuronales brindan hiperparámetros adicionales para permitirle especificar exactamente la tasa de este intervalo.

Por qué la deserción es capaz de disminuir el sobreajuste es una pregunta común. La respuesta es que la deserción puede reducir la posibilidad de que se desarrolle una codependencia entre dos neuronas. Dos neuronas que desarrollan una codependencia no podrán operar de manera efectiva cuando una se abandona. Como resultado, la red neuronal ya no puede depender de la presencia de cada neurona y se entrena en consecuencia. Esta característica disminuye su capacidad para memorizar la información que se le presenta, lo que obliga a la generalización.

La deserción también disminuye el sobreajuste al forzar un proceso de arranque en la red neuronal. Bootstrapping es una técnica de conjunto muy común. Discutiremos el ensamblaje con mayor detalle en el Capítulo 16, “Modelado con redes neuronales”. Básicamente, el ensamblaje es una técnica de aprendizaje automático que combina múltiples modelos para producir un mejor resultado que los logrados por modelos individuales. Conjunto es un término que se origina en los conjuntos musicales en los que el producto musical final que escucha el público es la combinación de muchos instrumentos.

Bootstrapping es una de las técnicas de conjunto más simples. El programador que usa bootstrapping simplemente entrena una serie de redes neuronales para realizar exactamente la misma tarea. Sin embargo, cada una de estas redes neuronales funcionará de manera diferente debido a algunas técnicas de entrenamiento y los números aleatorios utilizados en la inicialización del peso de la red neuronal. La diferencia de pesos provoca la variación del rendimiento. La salida de este conjunto de redes neuronales se convierte en la salida promedio de los miembros tomados en conjunto. Este proceso disminuye el sobreajuste a través del consenso de redes neuronales entrenadas de manera diferente.

La deserción funciona de forma similar al arranque. Puede pensar en cada red neuronal que resulta de un conjunto diferente de neuronas que se eliminan como un miembro individual en un conjunto. A medida que avanza el entrenamiento, el programa crea más redes neuronales de esta manera. Sin embargo, el abandono no requiere la misma cantidad de procesamiento que el arranque. Las nuevas redes neuronales creadas son temporales; existen solo para una iteración de entrenamiento. El resultado final también es una única red neuronal, en lugar de un conjunto de redes neuronales para promediar juntas.


In [1]:
import pandas as pd
from scipy.stats import zscore

# Read the data set
df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/jh-simple-dataset.csv",
    na_values=['NA','?'])

# Generar dummies para 'job'
df = pd.concat([df,pd.get_dummies(df['job'],prefix="job")],axis=1)
df.drop('job', axis=1, inplace=True)

# Generar dummies para 'area'
df = pd.concat([df,pd.get_dummies(df['area'],prefix="area")],axis=1)
df.drop('area', axis=1, inplace=True)

# Tratar Missing values en 'income'
med = df['income'].median()
df['income'] = df['income'].fillna(med)

# Standarizar rangos
df['income'] = zscore(df['income'])
df['aspect'] = zscore(df['aspect'])
df['save_rate'] = zscore(df['save_rate'])
df['age'] = zscore(df['age'])
df['subscriptions'] = zscore(df['subscriptions'])

# Convertir a numpy - Clasificación
x_columns = df.columns.drop('product').drop('id')
x = df[x_columns].values
dummies = pd.get_dummies(df['product']) # Clasificación
products = dummies.columns
y = dummies.values

Now we will see how to apply dropout to classification.

In [2]:
########################################
# Classification con dropout en Keras
########################################

import pandas as pd
import os
import numpy as np
from sklearn import metrics
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras import regularizers

# Cross-validate
kf = KFold(5, shuffle=True, random_state=42)
    
oos_y = []
oos_pred = []
fold = 0

for train, test in kf.split(x):
    fold+=1
    print(f"Fold #{fold}")
        
    x_train = x[train]
    y_train = y[train]
    x_test = x[test]
    y_test = y[test]
    
    #kernel_regularizer=regularizers.l2(0.01),
    
    model = Sequential()
    model.add(Dense(50, input_dim=x.shape[1], activation='relu')) # Oculta 1
    model.add(Dropout(0.5))
    model.add(Dense(25, activation='relu', \
                activity_regularizer=regularizers.l1(1e-4))) # Oculta 2
    # Por lo general, no agregue el abandono después de la capa oculta final
    #model.add(Dropout(0.5)) 
    model.add(Dense(y.shape[1],activation='softmax')) # Output
    model.compile(loss='categorical_crossentropy', optimizer='adam')

    model.fit(x_train,y_train,validation_data=(x_test,y_test),\
              verbose=0,epochs=500)
    
    pred = model.predict(x_test)
    
    oos_y.append(y_test)
    # probabilidades brutas a la clase elegida (probabilidad más alta)
    pred = np.argmax(pred,axis=1) 
    oos_pred.append(pred)        

    # fold's accuracy
    y_compare = np.argmax(y_test,axis=1) # accuracy
    score = metrics.accuracy_score(y_compare, pred)
    print(f"Fold score (accuracy): {score}")


# Cree la lista de predicción oos y calcule el error.
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
oos_y_compare = np.argmax(oos_y,axis=1) # accuracy

score = metrics.accuracy_score(oos_y_compare, oos_pred)
print(f"Final score (accuracy): {score}")    
    
# cross-validated prediction
oos_y = pd.DataFrame(oos_y)
oos_pred = pd.DataFrame(oos_pred)
oosDF = pd.concat( [df, oos_y, oos_pred],axis=1 )
#oosDF.to_csv(filename_write,index=False)

Fold #1
Fold score (accuracy): 0.66
Fold #2
Fold score (accuracy): 0.7375
Fold #3
Fold score (accuracy): 0.735
Fold #4
Fold score (accuracy): 0.6975
Fold #5
Fold score (accuracy): 0.69
Final score (accuracy): 0.704
